#### Step 1: Install TensorFlow and other required libraries

In [0]:
!pip install tensorflow

In [0]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout


#### Step 2: Load your dataset

In [0]:
mnist = fetch_openml('mnist_784')

In [0]:
type(mnist.data)

In [0]:
print('Dataset type:', type(mnist.data))

In [0]:
mnist.data.head()

#### Step 3: Convert the DataFrame into TensorFlow Dataset Object

In [0]:
def get_dataset(mnist):
    # Load MNIST dataset
    # mnist = fetch_openml('mnist_784')

    # Convert the data and target into numpy arrays
    X = mnist.data.astype('float32')
    y = mnist.target.astype('int32')

    # Normalize the data to have values between 0 and 1
    X /= 255.0

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Reshape the training data to 28x28x1 images
    X_train = X_train.values.reshape((-1, 28, 28, 1))
    X_test = X_test.values.reshape((-1, 28, 28, 1))

    # Create TensorFlow Dataset objects for training and testing sets
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

    # Print the shape of the training and testing sets
    print('Training data shape:', X_train.shape)
    print('Training labels shape:', y_train.shape)
    print('Testing data shape:', X_test.shape)
    print('Testing labels shape:', y_test.shape)

    return X_test, y_test, X_train, y_train


In [0]:
X_test, y_test, X_train, y_train = get_dataset(mnist)

#### Step 4: Define a deep learning mode

##### Option 1: Define a deep learning model from scratches

In [0]:
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10)
])

##### Option 2: Define a deep learning model from a pre-trained model

In [0]:
# Resize the input images to 224x224 and convert them to RGB with three channels
X_train = tf.image.grayscale_to_rgb(tf.image.resize(X_train, [224, 224]))
X_test = tf.image.grayscale_to_rgb(tf.image.resize(X_test, [224, 224]))

print('Training data shape:', X_train.shape)
print('Training labels shape:', y_train.shape)
print('Testing data shape:', X_test.shape)
print('Testing labels shape:', y_test.shape)

In [0]:
# Load the MobileNet model, excluding the top layer
base_model = MobileNet(include_top=False, input_shape=(224, 224, 3))

# Add a global average pooling layer and a fully connected output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)

# Combine the base model and the new layers to create the full model
model = tf.keras.models.Model(inputs=base_model.input, outputs=x)

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

#### Step 5: Compile the model

In [0]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#### Step 6: Train the model

In [0]:
# Fit the model to the training data
model.fit(X_train, y_train, batch_size=560, epochs=3, validation_data=(X_test, y_test))

#### Step 7: Evaluate the model

In [0]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test loss: ', test_loss)
print('Test Acc: ', test_acc)

#### Step 8: Make predictions

In [0]:
X_test[1].shape

In [0]:
np.reshape(X_test[1].shape, (1, 28, 28, 1))

In [0]:
# Make predictions on new data
# Make predictions on new data
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
print(y_pred_labels)

In [0]:
y_pred = model.predict(np.reshape(X_test[1], (1, 28, 28, 1)))

In [0]:
y_pred_labels = np.argmax(y_pred, axis=1)
y_pred_labels

In [0]:
 type(y_test)

In [0]:
y_test_array = y_test.values
# Get the label for the first example in the test set
label = y_test_array[0]

# Print the label
print("Label:", label)

In [0]:
def predict_and_compare(model, X_test, y_test, index):
    # Get the example from X_test at the given index
    example = X_test[index]

    # Reshape the example to the expected input shape
    example = np.reshape(example, (1, 28, 28, 1))

    # Predict the label for the example
    y_pred = model.predict(example)

    # Convert the predicted probabilities to class label
    y_pred_label = np.argmax(y_pred, axis=1)[0]

    # Get the true label from y_test using the index
    y_test_array = y_test.values
    # Get the label for the first example in the test set 
    y_true = y_test_array[index]

    # Print the predicted and true labels
    print("Predicted label:", y_pred_label)
    print("True label:", y_true)
    
    # Return the predicted and true labels
    return y_pred_label, y_true


In [0]:
# Predict and compare the label for the first example in the test set
y_pred_label, y_true = predict_and_compare(model, X_test, y_test, 0)